In [1]:
import os

In [2]:
%pwd

'd:\\a27_YEARS_OLD\\end_to_end_pro_chicken_disease_deepL\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\a27_YEARS_OLD\\end_to_end_pro_chicken_disease_deepL'

In [41]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

    loss: str
    metrics: list
    learning_rate: float

@dataclass
class PrepareCalbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [32]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [33]:
from pathlib import Path
import tensorflow as tf 

In [44]:
from cnnClassifier.config.configuration import PrepareCallbacksConfig
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir  =  os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)]
        )

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_image_size= params.IMAGE_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            loss = params.LOSS,
            metrics = params.METRICS,
            learning_rate = params.LEARNING_RATE
        )
        
        return training_config      

In [35]:
import time
import tensorflow as tf 
import urllib.request as request
from zipfile import ZipFile
import os
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )   
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [36]:
import os
import urllib.request as requeset
from zipfile import ZipFile
import tensorflow as tf
import time

In [37]:
from tensorflow.keras.optimizers import Adam

In [42]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Create a new optimizer instance
        optimizer = Adam()

        # Compile the model with the new optimizer
        self.model.compile(
            optimizer=optimizer, 
            loss=self.config.loss, 
            metrics=self.config.metrics
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [45]:
try:
    config = ConfigurationManager()

    #callback pipeline
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    #training pipeline
    Training_config = config.get_training_config()
    training = Training(config=Training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )

except Exception as e:
    raise e

[2024-09-11 01:26:44,449: INFO: common: yaml file: config\config.yaml loadded successfully]
[2024-09-11 01:26:44,453: INFO: common: yaml file: params.yaml loadded successfully]
[2024-09-11 01:26:44,455: INFO: common: created directory at: artifacts]
[2024-09-11 01:26:44,456: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-11 01:26:44,458: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-09-11 01:26:44,459: INFO: common: created directory at: artifacts\training]
[2024-09-11 01:26:44,711: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


d:\a27_YEARS_OLD\end_to_end_pro_chicken_disease_deepL\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 71s 4s/step - accuracy: 0.5994 - loss: 0.7312 - val_accuracy: 0.9219 - val_loss: 0.2795
[2024-09-11 01:27:56,260: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


In [ ]:
# 2:;18:45

#to run tensorboard
# tensorboard --logdir artifacts\prepare_callbacks\tensorboard_log_dir